In [ ]:
!pip install -U pyvisa pyvisa-py

## SNR estimation knowing emission frequencies and ocupations

The Keysight N9000B CXA Signal Analyzer can be used for signal-to-noise ratio (SNR) estimation across a wide range of applications in RF and wireless testing. This tool is suitable for essential signal characterization in both research and industrial environments.

### Typical SNR Estimation Process

Below are common steps and example approaches for estimating SNR using the N9000B CXA:

#### 1. **Signal Spectrum Measurement**

- **Connect** the device under test (DUT) to the RF input of the N9000B CXA.
- **Set frequency span and center frequency** to cover the signal of interest.
- **Capture** the signal using the analyzer’s spectrum display.

#### 2. **Measuring Signal Power**

- Use **marker functions** to identify the peak of the desired signal on the spectrum plot.
- **Read and log** the signal’s power in dBm directly from the marker readout at the carrier or main lobe.

#### 3. **Measuring Noise Power**

- **Set markers** in a nearby frequency region where no signal is present (only noise).
- **Use band power or noise marker tools** (if available) to measure the average noise floor level in dBm.

#### 4. **Calculating SNR**

- **Subtract** the measured noise power from the measured signal power (both in dB units):

  $$
  \mathrm{SNR\ (dB)} = P_{\text{signal}} - P_{\text{noise}}
  $$

- If your instrument provides “band/adjacent channel power” or “channel measurements”, it can directly report SNR for modulated signals over standard measurement bandwidths.

### Example: SNR Measurement Procedure

| Step            | Procedure with N9000B CXA                                     |
|-----------------|--------------------------------------------------------------|
| Setup           | Connect DUT to RF input, set center frequency and span.      |
| Signal Power    | Place marker at carrier frequency, read peak amplitude.      |
| Noise Power     | Place marker off-signal, use noise marker function, read value. |
| SNR Calculation | Compute difference: $$ \mathrm{SNR\ (dB)} = P_{\text{signal}} - P_{\text{noise}} $$ |

### Enhanced SNR Measurement Capabilities

- The N9000B CXA supports applications such as **analog/digital demodulation**, where **SNR can be measured automatically** along with other modulation quality indicators.
- When equipped with **vector signal analysis (VSA) software**, the analyzer allows for advanced SNR, Error Vector Magnitude (EVM), and adjacent channel measurements, providing direct SNR figures for complex signals like QAM, PSK, or OFDM.
- **Wide dynamic range** and low displayed average noise level (DANL) of the CXA facilitate SNR measurements even for weak signals[1][2].

### Practical Tips

- **Use narrow resolution bandwidth (RBW)** to improve noise power measurement accuracy.
- **Average multiple sweeps** to stabilize noise floor readings.
- **Apply preamplification** (if present) for low signal levels to enhance measurement sensitivity.

### Example Application Scenarios

- **Wireless transmitter testing:** SNR estimation over the main carrier to validate spectral purity and emission compliance.
- **EMI/EMC testing:** Characterizing SNR surrounding unwanted emissions to ensure standards compliance[3].
- **Lab education:** Demonstrating SNR fundamentals by measuring and comparing signal and noise in different RF scenarios[4][2].

### Links

[n900b Technical Suppport (Resources, manuals)](https://www.keysight.com/us/en/support/N9000B/cxa-signal-analyzer-multi-touch-9-khz-26-5-ghz.html#)

In [ ]:
import pyvisa
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
import os

# Instrument Params
N9000B_IP = '169.254.192.72'
VISA_TIMEOUT_MS = 5000  # Wait instrument response 5 seconds

# Connecting via LAN to n9000b instrument, and query spectrum trace (instantaneous)

In [ ]:
"""

try:
    # Init VISA
    rm = pyvisa.ResourceManager('@py')
    inst = rm.open_resource(f'TCPIP::{N9000B_IP}::INSTR')
    inst.timeout = VISA_TIMEOUT_MS

    # Identify
    print("Connecting Instrument...")
    idn = inst.query('*IDN?')
    print(f"Instrument: {idn.strip()} Connected...")

    # Clen instrument error Logs
    inst.write('*CLS')

    # Config Trace

    inst.write(':FREQ:CENT 785e6') 
    inst.write(':FREQ:SPAN 70e6') #bw
    inst.write(':BAND:RES 680e3') #Video bw (Just visualization)        

    # Obtain data of trace
    print("Init Trace Query...")
    inst.write(':FORM ASC')  # Ensure ASCII format
    trace_data = inst.query_ascii_values(':TRACe:DATA? TRACE1')
    print("Acquired Trace...")
    print("Shape trace:", np.shape(trace_data))

    plt.plot(trace_data)
    plt.title("n9000b Trace Data")
    plt.grid(True)
    plt.show()

except Exception as e:
    print(f"Error: {e}")

finally:
    if 'inst' in locals():
        inst.close()
        print("Conexión cerrada.")        

"""

# SNR M2M4 algorithm

El método M2M4 se basa en estadísticas de segundo y cuarto orden. En un canal con ruido blanco aditivo gaussiano (AWGN), los momentos impares de un proceso gaussiano son cero, mientras que los momentos pares no lo son.

Considerando una señal recibida \( x(n) \) compuesta por señal \( s(n) \) y ruido \( v(n) \), con medias cero y suponiendo independencia entre ambos:

\[
x(n) = s(n) + v(n)
\]

Los momentos de segundo y cuarto orden de la señal recibida se definen como:

\begin{align}
M_2 &= \mathbb{E}[|x(n)|^2] = \mathbb{E}[|s(n)|^2] + \mathbb{E}[|v(n)|^2] = P_s + \sigma_v^2 \\
M_4 &= \mathbb{E}[|x(n)|^4] = k_s P_s^2 + 4 P_s \sigma_v^2 + k_v \sigma_v^4
\end{align}

Donde:

- $P_s = \mathbb{E}[|s(n)|^2]$ es la potencia de la señal.
- $\sigma_v^2 = \mathbb{E}[|v(n)|^2] $ es la potencia del ruido.
- $ k_s = \dfrac{\mathbb{E}[|s(n)|^4]}{(\mathbb{E}[|s(n)|^2])^2} $
- $ k_v = \dfrac{\mathbb{E}[|v(n)|^4]}{(\mathbb{E}[|v(n)|^2])^2} $


Para canales complejos:
$
k_s = 2, \quad k_v = 2
$
Para canales reales:
$
k_s = 3, \quad k_v = 3
$

Resolviendo para la potencia de la señal $P_s$, se obtiene:

$$
\hat{P}_s = \frac{(k_v - 2)M_2^2 \pm \sqrt{(k_v - 2)^2 M_2^4 - 4(k_s - k_v)M_2^2 M_4 + 4(k_s - k_v)^2 M_4^2}}{2(k_s - k_v)}
$$

En la práctica, los momentos $ M_2 $ y $ M_4 $ se estiman mediante:

\begin{align}
\hat{M}_2 &= \frac{1}{N} \sum_{n=0}^{N-1} |x(n)|^2 \\
\hat{M}_4 &= \frac{1}{N} \sum_{n=0}^{N-1} |x(n)|^4
\end{align}

Finalmente, la estimación de la SNR es:

\begin{align}
\hat{N} &= \hat{M}_2 - \hat{P}_s \\
\hat{\rho} &= \frac{\hat{P}_s}{\hat{N}}
\end{align}

Para canales reales, la fórmula simplificada es:

$$
\hat{\rho}_{\text{real}} = \frac{M_2^2 - M_4}{M_4 - 3M_2^2}
$$

Para canales complejos:

$$
\hat{\rho}_{\text{complex}} = \frac{M_2^2 - M_4}{M_4 - 2M_2^2}
$$


In [ ]:
def m2m4_snr_estimator(i_vector, q_vector):
    """
    Estimates the Signal-to-Noise Ratio (SNR) using the M2M4 algorithm for complex signals.

    This function combines in-phase (I) and quadrature (Q) components to form a complex signal,
    then calculates the second and fourth order moments, and finally uses these moments
    to estimate the SNR.

    Args:
        i_vector (numpy.ndarray or list): A 1D array or list of in-phase components.
        q_vector (numpy.ndarray or list): A 1D array or list of quadrature components.

    Returns:
        float: The estimated SNR in decibels (dB). Returns NaN if the calculation is invalid
               (e.g., due to negative values under the square root).
    """
    i_vector = np.asarray(i_vector)
    q_vector = np.asarray(q_vector)

    if i_vector.shape != q_vector.shape:
        raise ValueError("I and Q vectors must have the same shape.")
    if i_vector.ndim > 1:
        raise ValueError("I and Q vectors must be 1-dimensional.")

    # 1. Combine I and Q vectors to form the complex signal y
    y = i_vector + 1j * q_vector

    # Calculate the squared magnitude of the complex signal
    abs_y_squared = np.abs(y)**2

    # 2. Calculate the second moment (M2)
    m2 = np.mean(abs_y_squared)

    # 3. Calculate the fourth moment (M4)
    m4 = np.mean(abs_y_squared**2)

    # 4. Apply the M2M4 formula
    # S = sqrt(2*M2^2 - M4)
    # N = M2 - S
    # SNR = S/N

    # Ensure the term inside the square root is non-negative
    discriminant = 2 * m2**2 - m4
    if discriminant < 0:
        # print("Warning: Discriminant for M2M4 is negative. Returning NaN.")
        return np.nan # Or handle as an error, indicating invalid input/estimation conditions

    estimated_signal_power_s = np.sqrt(discriminant)
    estimated_noise_power_n = m2 - estimated_signal_power_s

    # Avoid division by zero for noise power
    if estimated_noise_power_n <= 0:
        if estimated_signal_power_s > 0:
            return np.inf  # Very high SNR if noise is zero or negative
        else:
            return np.nan # Undefined if both signal and noise are zero/negative

    snr_linear = estimated_signal_power_s / estimated_noise_power_n

    # 5. Convert the SNR to dB
    snr_db = 10 * np.log10(snr_linear)

    return snr_db


# Load cs8 file from database
def load_cs8(filename):
    data = np.fromfile(filename, dtype=np.int8)
    I = data[0::2]  # Even samples as I
    Q = data[1::2]  # Odd samples as Q
    
    return I, Q

folder = "/home/javastral/Documents/data-jsongz" 
file_list = os.listdir(folder) #List folder path files

filename = os.path.join(folder, file_list[0]) # Concatenate paths

I, Q = load_cs8(filename)

m2m4_snrdB = m2m4_snr_estimator(I,Q)

print("M2M4 snr Estimation = ", m2m4_snrdB)

M2M4 snr Estimation =  5.478207951328404
